In [22]:
library(RPostgreSQL)
library(dplyr)
library(GenomicRanges)
library(doParallel)
library(dbplyr)

In [23]:
source("../BDDS/footprints/testdb/src/dbFunctions.R")

In [24]:
# connect to databases

# load chipseq traditionally -- will just put this in memory eventually
if(!exists("db.chipseq"))
    dbname="chipseq"
    port="5432"
    driver=dbDriver("PostgreSQL")
    user="trena"
    password="trena"
    host="whovian"
    db.chipseq <- dbConnect(drv=driver, user=user, password=password, dbname=dbname, host=host, port=port)

Here, we've connected to our database of ChipSeq data; it's our "gold standard" for comparison and it doesn't contain that many different different TFs.

## create map: M[TF] = c(motif1,motif2,...)

In [37]:
# chipseq hits table can fit in memory so load it up
chipseq.hits <- dbGetQuery(db.chipseq, "select * from hits")
chipseq.hits <- as.tbl(chipseq.hits)

# get chipseq regions data and change chrom notations from eg chr10 to just 10 as in fimo
chipseq.regions <- dbGetQuery(db.chipseq, "select * from regions")
chipseq.regions <- as.tbl(chipseq.regions)
chr.list <- chipseq.regions$chrom
cutoff <- nchar("chr")+1
no.chr.list <- substring(chr.list,cutoff)
chipseq.regions$chrom <- no.chr.list

Auto-disconnecting PostgreSQLConnection


In [38]:
# see which of the TFs in the chipseq data are mapped to any motifs
TF.motif.pairs <- read.csv("text_data_files/motif_to_tf_mappings_with_tfclass_include_multiple.csv",
                            stringsAsFactors=FALSE)

unique.cs.tfs <- unique(chipseq.hits$name)
cs.tf.nomatches <- unique.cs.tfs[!(unique(unique.cs.tfs) %in% unique(TF.motif.pairs$tfs))]
cs.tf.matches <- unique.cs.tfs[unique(unique.cs.tfs) %in% unique(TF.motif.pairs$tfs)]

In [39]:
length(unique.cs.tfs)
length(cs.tf.nomatches)
length(cs.tf.matches)

[1] 77

[1] 17

[1] 60

Up to this point we've done this:

1. Read all our hits from ChipSeq 
2. Read in all our Motif-TF mappings
3. Pulled out the unique TFs from ChipSeq (there are 77) and found those that are in our Motif-TF mapping (there are 60)

Now we want to use our motif-TF mapping to make a list of all TFs and their motifs AND to make a list of all motifs and their TFs. This will necessarily be 2 lists of lists; as Rory notes, a dictionary would be better, but this is R, not Python

In [28]:
# create a list of lists (poor sub for dict) for TFs -> motifs and motifs -> TFs
# only use jaspar motifs

TF.motif.pairs <- as.tbl(read.csv("text_data_files/motif_to_tf_mappings_with_tfclass_include_multiple.csv",
                            stringsAsFactors=FALSE))

TF.motif.pairs.jaspar <- subset(TF.motif.pairs, grepl("^MA[0-9]", motif) )

TFs.to.motifs <- list()
for (TF in cs.tf.matches) {
    this.TF.df <- subset(TF.motif.pairs.jaspar, tfs %in% TF)
    TFs.to.motifs[TF] <- list(this.TF.df$motif)
}
TFs.to.motifs <- TFs.to.motifs[lapply(TFs.to.motifs,length) >= 1]

motifs.to.TFs <- list()
for (mtf in unique(TF.motif.pairs.jaspar$motif)) {
    this.motif.df <- subset(TF.motif.pairs.jaspar, motif %in% mtf)
    motifs.to.TFs[mtf] <- list(this.motif.df$tfs)
}
motifs.to.TFs <- motifs.to.TFs[lapply(motifs.to.TFs,length) >= 1]

In [42]:
length(TFs.to.motifs)
length(motifs.to.TFs)

[1] 55

[1] 514

What this tells us is that when we look at only JASPAR motifs:

* There are only 55 TFs in our ChipSeq set that are JASPAR motifs
* There are 514 unique JASPAR motifs in our TF-motif mapping

We'll save these to a couple maps that we can load/use later. We also don't actually know how many motifs are mapped to by our TFs; we only know the total number of JASPAR motifs and the total number of TFs that have JASPAR motifs. 

We'll quickly run through our TF-to-motif mapping and see how many unique motifs are mapped to by our TFs

In [30]:
save(TFs.to.motifs, motifs.to.TFs, file="Rdata_files/Tfmotifmap.Rdata")
# load("Rdata_files/Tfmotifmap.Rdata")

In [31]:
allmots <- c()

for (TFname in names(TFs.to.motifs)) {
    allmots  <-  c(allmots, TFs.to.motifs[[TFname]])
}
length(unique(allmots))

[1] 195

In [32]:
# write.table(unique(allmots), file="text_data_files/unique_motifs_for_CS_TFs.txt", row.names=FALSE, col.names=FALSE, quote=FALSE)

So as we can summarize our "gold-standard" ChipSeq data now as follows:

* Our ChipSeq data contain 77 TFs, 60 of which we have a mapping for and 55 of which are JASPAR
* Mapping our 55, we can get 195 unique motifs

## function to output df containing pos and neg examples for each TF

Here's what we actually do for a given TF

1. We pull out all ChipSeq hits for the TF and their associated regions
2. Using our mapping, we look for ALL the occurences of the motifs for that TF in the FIMO database
3. We find the overlaps of (1) and (2), so now we have the regions in FIMO where there's overlap with ChipSeq hits. And this is for a given TF
4. Using our overlaps, we grab the FIMO motifs that are part of the overlaps; these are our "positives"
5. Using a ratio of neg/pos, we find the "positives" for each motif of the TF and multiply it by the ratio. So if we have neg/pos = 0.5 and there are 10 hits (positives) of a particular motif, we want 5 negatives. 
6. We then find the actual "negatives" for each motif by finding those that don't pop up in the positives. 
7. Using our "wanted" number of negatives and our "actual", we take the smaller of these 2 (so we get our wanted number, unless there's fewer than that) and then sample the negatives to grab this many for each motif. 
8. Finally, we put the positive and negative together, with a column called "cs_hit" designating a positive or negative

More succinctly, for each TF we:

1. Find all the motifs for a TF from our mapping.
2. Pull out all the places where these motifs in FIMO overlap the ChipSeq data
3. Compile a dataset of places where we had hits and then a sampling of the same motifs in FIMO where we didn't overlap ChipSeq data


loc,type,name,length,strand,sample_id,method,provenance,score1,score2,score3,score4,score5,score6
chr1:1677841-1677991,chipseq.peak,ATF2,151,NA,pooled,cusanovitch,chipseq.minid.tbd,155,NA,NA,NA,NA,NA
chr1:1678101-1678251,chipseq.peak,ATF2,151,NA,pooled,cusanovitch,chipseq.minid.tbd,131,NA,NA,NA,NA,NA
chr1:1828321-1828471,chipseq.peak,ATF2,151,NA,pooled,cusanovitch,chipseq.minid.tbd,30,NA,NA,NA,NA,NA
chr1:1828521-1828671,chipseq.peak,ATF2,151,NA,pooled,cusanovitch,chipseq.minid.tbd,53,NA,NA,NA,NA,NA
chr1:1828761-1828911,chipseq.peak,ATF2,151,NA,pooled,cusanovitch,chipseq.minid.tbd,30,NA,NA,NA,NA,NA
chr1:2255801-2255951,chipseq.peak,ATF2,151,NA,pooled,cusanovitch,chipseq.minid.tbd,243,NA,NA,NA,NA,NA


In [33]:
create.TF.df <- function(TF, neg.pos.ratio=10, verbose=FALSE) {
    
    # Note: this is how you make a DB connection in dplyr
    db.fimo.dplyr <- src_postgres(drv=dbDriver("PostgreSQL"),
                                  user="trena",
                                  password="trena",
                                  dbname="fimo",
                                  host="whovian",
                                  port="5432")
    tbl.fimo.dplyr <- tbl(db.fimo.dplyr, "fimo_hg38")
    
    # regions locs we can compute on but hits have TF info so need both
    # Here, we grab all hits for a TF, then grab the regions of those hits
    chipseq.hits.TF <- subset(chipseq.hits, name == TF)
    locs.TF <- chipseq.hits.TF$loc
    chipseq.regions.TF <- subset(chipseq.regions, loc %in% locs.TF)
    
    # next step is slow, gives you some context
    if (verbose == TRUE) {
        if (length(TFs.to.motifs[[TF]])==1) {
            message(paste(TF, "- querying fimo database for", length(TFs.to.motifs[[TF]]), "motif"))
        } else {
            message(paste(TF, "- querying fimo database for", length(TFs.to.motifs[[TF]]), "motifs"))
        }       
    }
        
    # this is the slow step -- doing SQL queries on tbl.fimo.dplyr = call to whole fimo database    
    # need branch since %in% conversion to SQL doesn't work on length == 1
    ## Basically: we find all instances of the TF's motif(s) in FIMO
    if (length(TFs.to.motifs[[TF]]) > 1 ) {
        fimo.motifs.for.TF <- as.tbl(as.data.frame(filter(tbl.fimo.dplyr, motifname %in% TFs.to.motifs[[TF]])))
    } else {
        fimo.motifs.for.TF <- as.tbl(as.data.frame(filter(tbl.fimo.dplyr, motifname  ==  TFs.to.motifs[[TF]])))
    }
    
    # find intersect using fast genomic ranges data structure
    # We make GR objects for the fimo motifs we just found and for the chipseq regions, then find their overlaps
    gr.fimo.TF <- with(fimo.motifs.for.TF, GRanges(chrom, IRanges(start=start, end=endpos)))
    gr.chipseq.TF <- with(chipseq.regions.TF, GRanges(chrom, IRanges(start=start, end=endpos)))
    overlaps.gr.TF <- findOverlaps(gr.chipseq.TF, gr.fimo.TF, type="any")
    overlaps.TF <- as.tbl(as.data.frame(overlaps.gr.TF))
    
    # row numbers in fimo.motifs.for.TF where motifs overlap with chipseq peaks
    positive.fimo.examples.rows.TF <- unique(overlaps.TF$subjectHits)
    positive.fimo.examples.TF.df <- fimo.motifs.for.TF[positive.fimo.examples.rows.TF,]
    
    # figure out how many negative samples for each motif we want
    tot.motif.counts.TF <- table(fimo.motifs.for.TF$motifname)
    pos.motif.counts.TF <- table(fimo.motifs.for.TF[positive.fimo.examples.rows.TF,]$motifname)
    nx.pos.motif.counts.TF <- pos.motif.counts.TF*neg.pos.ratio

    # want neg samples in fimo.motifs.for.TF to be non overlapping with chipseq peaks or pos examples
    neg.cands.for.single.TF.df <- subset(fimo.motifs.for.TF,
                                         !(start %in% positive.fimo.examples.TF.df$start) &
                                         !(endpos %in% positive.fimo.examples.TF.df$endpos))
    neg.motif.counts.TF <- table(neg.cands.for.single.TF.df$motifname)

    # don't try to sample more than the population
    neg.sample.counts.TF <- pmin(nx.pos.motif.counts.TF,neg.motif.counts.TF)
    
    # for each motif this TF matches, sample some examples where no CS hit
    negative.fimo.examples.TF.df <- tibble()
    for (motname in names(neg.sample.counts.TF)) {
        neg.cands.for.single.TF.df.single.motif <- subset(neg.cands.for.single.TF.df, motifname == motname)
        negative.fimo.examples.TF.df <- rbind(negative.fimo.examples.TF.df,
                                    sample_n(neg.cands.for.single.TF.df.single.motif, neg.sample.counts.TF[[motname]]))
    }
    
    # annotate and collect all samples
    positive.fimo.examples.TF.df <- as.tbl(cbind(positive.fimo.examples.TF.df, "cs_hit"=1))
    negative.fimo.examples.TF.df <- as.tbl(cbind(negative.fimo.examples.TF.df, "cs_hit"=0))
    all.fimo.examples.TF.df <- as.tbl(rbind(positive.fimo.examples.TF.df,negative.fimo.examples.TF.df))
    
    return(all.fimo.examples.TF.df)
    
}

## create df of pos/neg samples for all TFs all together

With our function defined, we'll now pull out the positive and negative samples for each TF; we're doing so with a neg/pos ration of 9. We're also doing this using a parallel strategy. The output gets saved in an `RData` file

In [34]:
# ratio of negative to positve examples in data set
pnr=9

In [35]:
# make a cluster for parallel computing
cl <- makePSOCKcluster(11)
clusterEvalQ(cl, {
    library(DBI)
    library(RPostgreSQL)
    library(dplyr)
    library(GenomicRanges)
})
registerDoParallel(cl)

# parallel loop over all TFs
sorted.TF.names <- sort(names(TFs.to.motifs))
N.TF <- length(sorted.TF.names)

all.TF.df <- foreach(i.TF=1:N.TF,
                     .inorder=FALSE,
                     .combine=rbind,
                     .packages=c("DBI", "RPostgreSQL", "dplyr", "GenomicRanges")) %dopar% {    
    TFname <- sorted.TF.names[[i.TF]]
    create.TF.df(TFname, neg.pos.ratio=pnr, verbose=TRUE)
}

# clean up after parallel setup
stopCluster(cl)
registerDoSEQ()

# save data
fname=paste("/ssd/mrichard/data/all.TF.fimo.samples.ratio.",pnr,".df.RData", sep="")
save(all.TF.df, file=fname)

[[1]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[2]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[3]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[4]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[5]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[6]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[7]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[8]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[9]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[10]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"         

[[11]]
 [1] "GenomicRanges" "GenomeInfoDb"  "IRanges"       "S4Vectors"    
 [5] "BiocGenerics"  "parallel"      "stats4"        "dplyr"        
 [9] "RPostgreSQL"   "DBI"           "stats"         "graphics"     
[13] "grDevices"     "utils"         "datasets"      "methods"      
[17] "base"

Warning message:
“closing unused connection 14 (<-localhost:11087)”Warning message:
“closing unused connection 13 (<-localhost:11087)”Warning message:
“closing unused connection 12 (<-localhost:11087)”Warning message:
“closing unused connection 11 (<-localhost:11087)”Warning message:
“closing unused connection 10 (<-localhost:11087)”Warning message:
“closing unused connection 9 (<-localhost:11087)”Warning message:
“closing unused connection 8 (<-localhost:11087)”Warning message:
“closing unused connection 7 (<-localhost:11087)”Warning message:
“closing unused connection 6 (<-localhost:11087)”Warning message:
“closing unused connection 5 (<-localhost:11087)”Warning message:
“closing unused connection 4 (<-localhost:11087)”

## serial version of production loop

This is the same thing, but done serially instead of with parallel computing. Not much to see here. 

In [36]:
# serial version -- hung on last TF, don't know why

all.TF.df <- tibble()

sorted.TF.names <- sort(names(TFs.to.motifs))
for (TFname in sorted.TF.names) {
    
    TFnum <- which(sorted.TF.names %in% TFname)
    message(paste("Processing TF", TFnum,"/", length(sorted.TF.names)))
    TF.df <- create.TF.df(TFname, neg.pos.ratio=9, verbose=TRUE)
    all.TF.df <- rbind(all.TF.df,TF.df)
    
    save(all.TF.df, file="/ssd/mrichard/data/all.TF.fimo.samples.ratio.9.df.RData")

}

Processing TF 1 / 55
ATF2 - querying fimo database for 8 motifs
Processing TF 2 / 55
ATF3 - querying fimo database for 3 motifs
Processing TF 3 / 55
BATF - querying fimo database for 2 motifs
Processing TF 4 / 55
BHLHE40 - querying fimo database for 1 motif
Processing TF 5 / 55
CEBPB - querying fimo database for 1 motif
Processing TF 6 / 55
CTCF - querying fimo database for 1 motif
Auto-disconnecting PostgreSQLConnection
Auto-disconnecting PostgreSQLConnection
Auto-disconnecting PostgreSQLConnection
Processing TF 7 / 55
E2F4 - querying fimo database for 7 motifs
Processing TF 8 / 55
EBF1 - querying fimo database for 1 motif
Processing TF 9 / 55
EGR1 - querying fimo database for 16 motifs
Auto-disconnecting PostgreSQLConnection
Auto-disconnecting PostgreSQLConnection
Auto-disconnecting PostgreSQLConnection
Auto-disconnecting PostgreSQLConnection
Auto-disconnecting PostgreSQLConnection
Processing TF 10 / 55
ELF1 - querying fimo database for 21 motifs
Auto-disconnecting PostgreSQLConnecti